# GSM-VED: Visco-elasto-damage 

In [ ]:
%matplotlib widget
from bmcs_matmod.gsm.gsm_models import GSM1D_VED
import matplotlib.pylab as plt
from IPython.display import display, Markdown
import sympy as sp
import numpy as np
sp.init_printing()

In [ ]:
gsm = GSM1D_VED()
Markdown(gsm.latex_potentials())

In [ ]:
mparams = dict(E=1, eta_ve=1e+2, c=1, r=1, S=0.1, eps_0=1.0)
n_t = 100
n_I = 1
eps_max = 4
t_t = np.linspace(0, 100, n_t)
eps_n_t = np.linspace(0, eps_max, n_t)
t_t, eps_ta, sig_ta, Eps_t, Sig_t, iter_t, lam_t, (d_t_t, d_eps_ta) = gsm.get_response(eps_n_t, t_t, **mparams )
# Plot viscoplastic strain and damage along time
eps_t = eps_ta[:, 0]
eps_v_t = Eps_t[:, 0, 0]
omega_t = Eps_t[:, 0, 1]

In [ ]:
fig, (ax_sig, ax_omega) = plt.subplots(1, 2, figsize=(12, 4))

# Plot stress-strain and iterations-strain curves
ax_iter = ax_sig.twinx()
ax_eps_v = ax_omega.twinx()

ax_sig.plot(eps_t, Sig_t[:,:,0], color='cadetblue', label=r'$\varepsilon$')
ax_sig.plot(eps_t-eps_v_t, Sig_t[:,:,0], ls='dashed', color='cadetblue', label=r'$\varepsilon-\varepsilon^\mathrm{ve}$')
ax_sig.set_ylabel(r'$\sigma$', color='darkslategray')
ax_sig.set_xlabel(r'$\varepsilon$')
ax_sig.legend()

ax_iter.plot(eps_t, iter_t, ls='dashed', color='gray')
ax_iter.set_ylabel(r'$k$', color='gray')

ax_omega.plot(t_t, omega_t, label='Damage', color='royalblue')
ax_eps_v.plot(t_t, eps_v_t, label='Viscoplastic Strain', color='orange')
ax_eps_v.plot(t_t, eps_t, label='Total Strain', color='black')

ax_omega.set_xlabel(r'$t$')
ax_omega.set_ylabel(r'$\omega$', color='royalblue')
ax_eps_v.set_ylabel(r'$\varepsilon_\mathrm{ve}$', color='orange')

plt.tight_layout()
plt.show()

In [ ]:
%matplotlib widget
mparams = dict(E=1, eta_ve=1e+2, c=1, r=1, S=0.01, eps_0=0)
t_, eps_dot_, t_1_, t_2_, t3_ = sp.symbols('t eps_dot t_1 t_2 t_3', real=True, positive=True)
eps_t_ = sp.Piecewise(
    (eps_dot_ * t_, t_ < t_1_),
    (eps_dot_ * t_1_, t_ < t_2_),
    (eps_dot_ * t_1_ - eps_dot_*(t_ - t_2_), True)
)
get_eps_t = sp.lambdify((t_, eps_dot_, t_1_, t_2_, t3_), eps_t_, 'numpy')
final_time = 300
eps_max = 2.5
dot_eps_t = eps_max / final_time * 3
t_t = np.linspace(0, final_time, 1000)
eps_n_t = get_eps_t(t_t, dot_eps_t, 0.25*final_time, 0.75*final_time, final_time)
fig, ax = plt.subplots(1,1)
ax.plot(t_t, eps_n_t)

In [ ]:

t_t, eps_ta, sig_ta, Eps_t, Sig_t, iter_t, lam_t, (d_t_t, d_eps_ta) = gsm.get_response(eps_n_t, t_t, **mparams)
# Plot viscoplastic strain and damage along time
eps_t = eps_ta[:, 0]
eps_v_t, omega_t, z_t = Eps_t[:, 0, :].T
sig_v_t, Y_t, Z_t = Sig_t[:, 0, :].T

In [ ]:
fig, (ax_sig, ax_omega) = plt.subplots(1, 2, figsize=(12, 4))

# Plot stress-strain and iterations-strain curves
ax_iter = ax_sig.twinx()
ax_eps = ax_omega.twinx()

ax_sig.plot(eps_t, sig_v_t, color='cadetblue', label=r'$\varepsilon$')
ax_sig.plot(eps_t-eps_v_t, Sig_t[:,:,0], ls='dashed', color='cadetblue', label=r'$\varepsilon-\varepsilon^\mathrm{ve}$')
ax_sig.set_ylabel(r'$\sigma$', color='darkslategray')
ax_sig.set_xlabel(r'$\varepsilon$')
ax_sig.legend()

ax_iter.plot(eps_t, iter_t, ls='dashed', color='gray')
ax_iter.set_ylabel(r'$k$', color='gray')

ax_omega.plot(t_t, omega_t, label='Damage', color='brown')
ax_eps.plot(t_t, eps_t, label=r'$\varepsilon$', color='cadetblue', lw=1)
ax_eps.plot(t_t, eps_v_t, label=r'$\varepsilon_\mathrm{vp}$', color='cadetblue', ls='dashed', lw=1)
#ax_eps.legend()
ax_omega.set_xlabel(r'$t$')
ax_omega.set_ylabel(r'$\omega$', color='brown')
ax_eps.set_ylabel(r'$\varepsilon$', color='cadetblue')

plt.tight_layout()
plt.show()